In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

#### Function that loads a document

In [3]:
def load_document(file):
    from langchain.document_loaders import PyPDFLoader
    print(f'Loading {file}...')
    loader = PyPDFLoader(file)
    data = loader.load()
    return data

#### Load a pdf file data

In [6]:
pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/74/a9/5ccde1312650dd03e65350224fea85d9a430c182a01f056599cbb76f7390/pypdf-3.17.0-py3-none-any.whl.metadata
  Using cached pypdf-3.17.0-py3-none-any.whl.metadata (7.5 kB)
Using cached pypdf-3.17.0-py3-none-any.whl (277 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
data = load_document('./files/us_constitution.pdf')
print(data[0].page_content)

Loading ./files/us_constitution.pdf...
The
United
States
Constitution
W e
the
People
of
the
United
States,
in
Order
to
form
a
more
perfect
Union,
establish
Justice,
insure
domestic
T ranquility ,
provide
for
the
common
defence,
promote
the
general
W elfare,
and
secure
the
Blessings
of
Liberty
to
ourselves
and
our
Posterity ,
do
ordain
and
establish
this
Constitution
for
the
United
States
of
America.
The
Constitutional
Con v ention
Article
I
Section
1:
Congress
All
legislative
Powers
herein
granted
shall
be
vested
in
a
Congress
of
the
United
States,
which
shall
consist
of
a
Senate
and
House
of
Representatives.
Section
2:
The
House
of
Representatives


In [8]:
print(data[0].metadata)

{'source': './files/us_constitution.pdf', 'page': 0}


In [9]:
print(f'You have {len(data)} pages in your pdf')

You have 41 pages in your pdf


#### Improve load document function to also load a docx file

In [13]:
pip install docx2txt

  Using cached docx2txt-0.8-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)
    print(f'Loading {file}...')
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        loader = Docx2txtLoader(file)
    else:
        print('Document format is not supported')
        return None
    
    data = loader.load()
    return data

In [16]:
data = load_document('files/the_great_gatsby.docx')
print(data[0].metadata)

Loading files/the_great_gatsby.docx...
{'source': 'files/the_great_gatsby.docx'}


In [15]:
print(len(data))

1


### Loading from Wikipedia

In [18]:
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [20]:
pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
data = load_from_wikipedia('GPT-4')

In [25]:
print(data[0].page_content)

Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its series of GPT foundation models. It was initially released on March 14, 2023, and has been made publicly available via the paid chatbot product ChatGPT Plus, and via OpenAI's API.  As a transformer-based model, GPT-4 uses a paradigm where pre-training using both public data and "data licensed from third-party providers" is used to predict the next token. After this step, the model was then fine-tuned with reinforcement learning feedback from humans and AI for human alignment and policy compliance.: 2 Observers reported that the iteration of ChatGPT using GPT-4 was an improvement on the previous iteration based on GPT-3.5, with the caveat that GPT-4 retains some of the problems with earlier revisions. GPT-4 is also capable of taking images as input on ChatGPT. OpenAI has declined to reveal various technical details and statistics about GPT-4, such as the precise siz

In [26]:
data = load_from_wikipedia('GPT-4', 'fr')
print(data[0].page_content)

GPT-4 (acronyme de Generative Pre-trained Transformer 4) est un modèle de langage multimodalmultimodal, de type transformeur génératif pré-entraîné, développé par la société OpenAI et sorti le 14 mars 2023, il succède à GPT-3.


== Caractéristiques ==
OpenAI annonce, sur son blog, GPT-4 comme étant « plus fiable, créatif et capable de gérer des instructions beaucoup plus nuancées que GPT-3.5 ». L'organisation a produit deux versions de GPT-4 avec des fenêtres contextuelles de 8 192 et 32 768 jetons, une amélioration significative par rapport à GPT-3.5 et GPT-3, qui étaient limités à 4 096 et 2 048 jetons respectivement. Contrairement à son prédécesseur, GPT-4 peut prendre des images ainsi que du texte comme entrées.
OpenAI adopte une approche fermée en ce qui concerne les détails techniques de GPT-4 ; le rapport technique s'est explicitement abstenu de spécifier la taille, l'architecture, le matériel ou la méthode de formation du modèle. Le rapport affirme que « le paysage concurrentie

#### Chunking

In [28]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = splitter.split_documents(data)
    return chunks

In [29]:
chunks = chunk_data(data)
print(len(chunks))

44


#### Calculating embedding cost

In [30]:
pip install tiktoken


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [57]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

In [58]:
print_embedding_cost(chunks)

Total tokens: 2079
Embedding Cost in USD: 0.000832
